Question 3-Vehicle Routing Problem (VRP):

You are a delivery manager for a small company with a single depot and a fleet of three delivery trucks. There are seven customer locations that need to be visited for deliveries. Each customer has a known demand for your product, and each truck has a maximum capacity. Your goal is to find an optimal route for each truck to serve all customers while minimizing the total distance traveled.

Information:

Depot Location: (0, 0)/
Customer Locations:
Customer 1: (1, 2) with a demand of 3 units./
Customer 2: (2, 4) with a demand of 4 units./
Customer 3: (3, 5) with a demand of 2 units./
Customer 4: (5, 7) with a demand of 3 units./
Customer 5: (6, 8) with a demand of 5 units./
Customer 6: (8, 5) with a demand of 2 units./
Customer 7: (9, 2) with a demand of 4 units./

Truck Capacities: Each truck can carry up to 6 units.

Traveling between locations follows the Euclidean distance.


Your task is to find the optimal routes for the three trucks to serve all customers, taking into account the truck capacities and minimizing the total distance traveled.

In [8]:
import cplex
from docplex.mp.model import Model

# Model
model = Model(name='Vehicle_Routing_Problem')

# the customer locations and demands, including the depot (customer 0)
customers = {
    0: (0, 0, 0),  
    1: (1, 2, 3),
    2: (2, 4, 4),
    3: (3, 5, 2),
    4: (5, 7, 3),
    5: (6, 8, 5),
    6: (8, 5, 2),
    7: (9, 2, 4),
}

# the number of trucks and their capacity
num_trucks = 3
truck_capacity = 6

# Decision variables for each edge
x = {(i, j): model.binary_var(name=f'x_{i}_{j}') for i in customers for j in customers if i != j}
u = {i: model.integer_var(lb=0, ub=truck_capacity, name=f'u_{i}') for i in customers if i != 0}

# Objective: Minimize the total distance
model.minimize(model.sum(x[i, j] * ((customers[i][0] - customers[j][0]) ** 2 + (customers[i][1] - customers[j][1]) ** 2) ** 0.5 for i in customers for j in customers if i != j))

# Constraints
for i in customers:
    if i != 0:  # Exclude the depot
        model.add_constraint(u[i] >= customers[i][2], ctname=f'Demand_{i}')
        model.add_constraint(u[i] <= truck_capacity, ctname=f'Capacity_{i}')

# Subtour elimination
for i in customers:
    for j in customers:
        if i != j and i != 0 and j != 0:
            model.add_constraint(u[i] - u[j] + (truck_capacity - 1) * x[i, j] <= truck_capacity - 1, ctname=f'SubTour_{i}_{j}')

# Set the maximum number of trucks used
model.add_constraint(model.sum(x[0, j] for j in customers if j != 0) == num_trucks, ctname='Trucks')

# Solve
model.solve()

# Results
for i in customers:
    for j in customers:
        if i != j and x[i, j].solution_value == 1:
            print(f'Truck travels from customer {i} to customer {j}')

# Total distance traveled
print(f'Total Distance: {model.objective_value}')


Truck travels from customer 0 to customer 1
Truck travels from customer 0 to customer 2
Truck travels from customer 0 to customer 3
Total Distance: 12.53915582734467
